In [1]:
!nvidia-smi 

Fri Aug 26 15:24:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers -q

     |████████████████████████████████| 4.7 MB 34.5 MB/s 
     |████████████████████████████████| 6.6 MB 73.7 MB/s 
     |████████████████████████████████| 120 kB 70.6 MB/s 


In [3]:
!pip install datasets -q

     |████████████████████████████████| 365 kB 30.6 MB/s 
     |████████████████████████████████| 115 kB 50.5 MB/s 
     |████████████████████████████████| 212 kB 74.2 MB/s 
     |████████████████████████████████| 127 kB 71.8 MB/s 


##Creating Data

In [4]:
!wget https://raw.githubusercontent.com/csebuetnlp/banglabert/master/question_answering/sample_inputs/train.json

--2022-08-26 15:24:20--  https://raw.githubusercontent.com/csebuetnlp/banglabert/master/question_answering/sample_inputs/train.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4562448 (4.4M) [text/plain]
Saving to: ‘train.json’

train.json          100%[===================>]   4.35M  --.-KB/s    in 0.02s   

2022-08-26 15:24:23 (235 MB/s) - ‘train.json’ saved [4562448/4562448]



In [5]:
!wget https://raw.githubusercontent.com/csebuetnlp/banglabert/master/question_answering/sample_inputs/test.json

--2022-08-26 15:24:23--  https://raw.githubusercontent.com/csebuetnlp/banglabert/master/question_answering/sample_inputs/test.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4562448 (4.4M) [text/plain]
Saving to: ‘test.json’

test.json           100%[===================>]   4.35M  --.-KB/s    in 0.02s   

2022-08-26 15:24:24 (219 MB/s) - ‘test.json’ saved [4562448/4562448]



In [6]:
!wget https://raw.githubusercontent.com/csebuetnlp/banglabert/master/question_answering/sample_inputs/validation.json

--2022-08-26 15:24:24--  https://raw.githubusercontent.com/csebuetnlp/banglabert/master/question_answering/sample_inputs/validation.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4562448 (4.4M) [text/plain]
Saving to: ‘validation.json’

validation.json     100%[===================>]   4.35M  --.-KB/s    in 0.01s   

2022-08-26 15:24:27 (308 MB/s) - ‘validation.json’ saved [4562448/4562448]



In [7]:
import json
from datasets import load_dataset



def convert_to_jsonl(input_filename, output_filename):

  with open(input_filename) as f:
      dataset = json.load(f)

  with open(output_filename, "w") as f:
      for article in dataset["data"]:
          title = article["title"]
          for paragraph in article["paragraphs"]:
              context = paragraph["context"]
              answers = {}
              for qa in paragraph["qas"]:
                  question = qa["question"]
                  idx = qa["id"]
                  answers["text"] = [a["text"] for a in qa["answers"]]
                  answers["answer_start"] = [a["answer_start"] for a in qa["answers"]]
                  f.write(
                      json.dumps(
                          {
                              "id": idx,
                              "title": title,
                              "context": context,
                              "question": question,
                              "answers": answers,
                          }
                      )
                  )
                  f.write("\n")



In [8]:
convert_to_jsonl("train.json","train.jsonl")

In [9]:
convert_to_jsonl("test.json","test.jsonl")

In [10]:
convert_to_jsonl("validation.json","validaiton.jsonl")

In [11]:
from dataclasses import field
from datasets import load_dataset

data_files = {
    "train":"train.jsonl",
    "test":"test.jsonl",
    "validation":"validaiton.jsonl"
}

raw_datasets = load_dataset("json",data_files = data_files)
raw_datasets

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-97c960d33b33b48a/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 2504
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 2504
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 2504
    })
})

In [12]:
raw_datasets["train"][2]

{'id': 'bengali--981248442377505718-0-2649',
 'title': 'শেখ মুজিবুর রহমান',
 'context': 'শেখ মুজিবুর রহমান (১৭ মার্চ ১৯২০ - ১৫ আগস্ট ১৯৭৫) বাংলাদেশের প্রথম রাষ্ট্রপতি ও ভারতীয় উপমহাদেশের একজন অন্যতম প্রভাবশালী রাজনৈতিক ব্যক্তিত্ব যিনি বাঙালীর অধিকার রক্ষায় ব্রিটিশ ভারত থেকে ভারত বিভাজন আন্দোলন এবং পরবর্তীতে  পূর্ব পাকিস্তান থেকে বাংলাদেশ প্রতিষ্ঠার সংগ্রামে নেতৃত্ব প্রদান করেন। প্রাচীন বাঙ্গালি সভ্যতার আধুনিক স্থপতি হিসাবে শেখ মুজিবুর রহমানকে বাংলাদেশের জাতির জনক বা জাতির পিতা বলা হয়ে থাকে। তিনি মাওলানা আব্দুল হামিদ খান ভাসানী প্রতিষ্ঠিত আওয়ামী লীগের সভাপতি, বাংলাদেশের প্রথম রাষ্ট্রপতি এবং পরবর্তীতে এদেশের প্রধানমন্ত্রীর দায়িত্ব পালন করেন। জনসাধারণের কাছে তিনি শেখ মুজিব এবং শেখ সাহেব হিসাবে বেশি পরিচিত এবং তার উপাধি বঙ্গবন্ধু। তার কন্যা শেখ হাসিনা বাংলাদেশ আওয়ামী লীগের বর্তমান সভানেত্রী এবং বাংলাদেশের বর্তমান প্রধানমন্ত্রী।',
 'question': 'শেখ মুজিবুর রহমান কবে জন্মগ্রহণ করেন ?',
 'answers': {'text': ['১৭ মার্চ ১৯২০'], 'answer_start': [19]}}

In [13]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(
                lambda x: [typ.feature.names[i] for i in x]
            )
    display(HTML(df.to_html()))

In [14]:
show_random_elements(raw_datasets["train"])

,id,title,context,question,answers
0,bengali--6853666904033538882-9-3401,ত্রিপুরা,"ত্রিপুরার প্রধান রাজনৈতিক জোট ও দলগুলি হল ভারতীয় জনতা পার্টি, বামফ্রন্ট এবং ভারতীয় জাতীয় কংগ্রেস। বর্তমানে ত্রিপুরা সরকারে ক্ষমতাসীন রয়েছে মুখ্যমন্ত্রী বিপ্লব কুমার দেব নেতৃত্বাধীন বিজেপি। ১৯৭৭ সালের পূর্বাবধি ত্রিপুরায় ক্ষমতাসীন ছিল ভারতীয় জাতীয় কংগ্রেস। ১৯৭৭ থেকে ১৯৮৮ ত্রিপুরা সরকার পরিচালিত হয় বামফ্রন্টের নেতৃত্বাধীনে এবং ১৯৯৩ সাল থেকে আবার তারা ক্ষমতায় প্রত্যাবর্তন করে। ১৯৮৮ থেকে ১৯৯৩ পর্যন্ত ভারতীয় জাতীয় কংগ্রেস এবং ত্রিপুরা উপজাতি যুব সমিতির জোট সরকার পরিচালনা করে। ১৬ ফেব্রুয়ারি ২০১৮ তারিখে অনুষ্ঠিত ত্রিপুরার বিধানসভা নির্বাচনে ৬০টি আসনের ৪৪টি তে জয়লাভ করে ভারতীয় জনতা পার্টি অধীন এন.ডি.এ. জোট ক্ষমতাসীন হয় অপরদিকে সিপিআই(এম) পায় মাত্র ১৬টি আসন। ভারতীয় জাতীয় কংগ্রেস এর ৫৫টি আসনে জামানত জব্দ হয়।",ত্রিপুরার বর্তমান (২০১৯) মুখ্যমন্ত্রী কে ?,"{'text': ['বিপ্লব কুমার দেব'], 'answer_start': [156]}"
1,bengali--5462601724301448508-7-3975,খান আতাউর রহমান,"তার প্রথম পরিচালিত চলচ্চিত্র অনেক দিনের চেনা। ছায়াছবিটি ১৯৬৩ সালে মুক্তি পায়।[11] ১৯৬৭ সালে তিনি বাংলার শেষ স্বাধীন নবাব সিরাজউদ্দৌলার জীবনী নিয়ে নির্মাণ করেন নবাব সিরাজউদ্দৌল্লা। চলচ্চিত্রটি ১৯৬৯ সালে মস্কো আন্তর্জাতিক চলচ্চিত্র উৎসব-এ প্রদর্শিত হয় এবং শ্রেষ্ঠ চলচ্চিত্র হিসেবে গোল্ডেন প্রাইজের জন্য মনোনীত হয়। এরপর তিনি নির্মাণ করেন সাত ভাই চম্পা, অরুণ বরুণ কিরণমালা, জোয়ার ভাটা। মুক্তিযুদ্ধের পর ১৯৭৩ সালে নির্মাণ করেন আবার তোরা মানুষ হ ; যার বিষয়বস্তু ছিল যুদ্ধ পরবর্তী বাস্তবতা। ১৯৭৫ সালে প্রমোদ কর ছদ্মনামে গ্রামীণ পটভূমিতে নির্মাণ করেন রোমান্টিক চলচ্চিত্র সুজন সখী। এই চলচ্চিত্রের জন্য তিনি ১ম জাতীয় চলচ্চিত্র পুরস্কারে শ্রেষ্ঠ চিত্রনাট্যকার হিসেবে পুরস্কৃত হয়। ৮০'র দশকের নির্মাণ করেন হিসাব নিকাশ এবং পরশপাথর নামের দুইটি ছায়াছবি।[12] মুক্তিযুদ্ধের উপর ১৯৯৪ সালে তিনি এখনো অনেক রাত চলচ্চিত্র নির্মাণের কাজ শুরু করেন। ১৯৯৭ সালে ছবির কাজ শেষ হয়। কিন্তু সেন্সর বোর্ড ছবির ৭টি স্থানে দৃশ্য কেটে ফেলার নির্দেশ দেয়ায় ক্ষুব্ধ হন তিনি।[13]",খান আতাউর রহমানের নির্মিত প্রথম চলচ্চিত্রের নাম কী ?,"{'text': ['অনেক দিনের চেনা'], 'answer_start': [29]}"
2,bengali--6188005353247118165-0-4117,মস্তিষ্ক,"মস্তিষ্ক হল কেন্দ্রীয় স্নায়ুতন্ত্রের একটি অপরিহার্য অংশ, যা করোটির অভ্যন্তরে অবস্থিত এবং দেহের প্রধান নিয়ন্ত্রণকেন্দ্র।ভ্রুণ অবস্থায় সুষুম্নাকান্ডের অগ্রবর্তী দন্ডাকার অংশ ভাঁজ হয়ে পর পর ৩টি বিষমাকৃতির স্ফীতি তৈরী করে ৷ স্ফীতি ৩টি মিলেই গঠিত হয় মস্তিষ্ক ৷ প্রাপ্তবয়স্ক লোকের মস্তিষ্কের আয়তন ১৫০০ ঘন সেন্টিমিটার, গড় ওজন ১.৩৬ কেজি এবং এতে প্রায় ১০ বিলিয়ন নিউরন থাকে ৷",একটি প্রাপ্ত বয়স্ক মানব মস্তিষ্কের গড় আয়তন কত ?,"{'text': ['১৫০০ ঘন সেন্টিমিটার'], 'answer_start': [300]}"
3,bengali-7684827071350169783-7-2993,শ্রীলঙ্কা,"শ্রীলঙ্কার সংবিধান অনুযায়ী দেশটি গণতান্ত্রিক, সামজতান্ত্রিক, প্রজাতন্ত্রী হিসেবে পরিচিত হবে, দেশটিকে একেশ্বরবাদী রাষ্ট্রও বলা হয়েছে। সরকার ব্যবস্থা গড়ে উঠেছে সংসদীয় ও রাষ্ট্রপতি শাসিত ব্যবস্থার সমন্বয়ে। শ্রীলঙ্কার রাষ্ট্রপতি একাধারে রাষ্ট্রপ্রধান, সামরিক বাহিনীর প্রধান প্রশাসক ও সরকার প্রধান এবং তিনি নির্বাচিত হন ছয় বছরের জন্য। রাষ্ট্রপতি দেশের সংসদ এবং ২২৫ সদস্যের আইন প্রণয়নকারী পরিষদের কাজে দায়বদ্ধ। রাষ্ট্রপতি নির্বাচিত সংসদ সদস্যের মধ্য থেকে একজনকে মন্ত্রী সভার প্রধান হিসেবে নিয়োগ দেন। প্রধানমন্ত্রী রাষ্ট্রপতির ডেপুটি হিসেবে কাজ করেন এবং সংসদের সরকারি দলের নেতৃত্ব দেন। প্রতিটি জেলা হতে সংসদ সদস্যরা সার্বজনীন ভোটে নির্বাচিত হন। রাষ্ট্রপতি সংসদের একবছর কার্যক্রম অতিবাহিত হবার পর সংসদ স্থগিত অথবা সমাপ্তি ঘোষনা করতে পারেন। সংসদ সকল প্রকার আইন প্রণয়ন করার ক্ষমতা রাখে। শ্রীমাভো বন্দেরনায়েক শ্রীলঙ্কার নির্বাচিত প্রথম মহিলা প্রধানমন্ত্রী। তাঁর সুযোগ্যকন্যা চন্দ্রিকা কুমারাতুঙ্গা ১৯৯৪ থেকে ২০০৫ সাল পর্যন্ত রাষ্ট্রপতি হবার আগে ১৯৯৪ সালের আগষ্ট থেকে ডিসেম্বর পর্যন্ত প্রধানমন্ত্রীও ছিলেন। মৈত্রীপাল সিরিসেন ও রানিল বিক্রমাসিংহে ৯ জানুয়ারি, ২০১৫ তারিখে রাষ্ট্রপতি ও প্রধানমন্ত্রী নির্বাচিত হন।",সিনাই উপদ্বীপে বসবাসকারী মানুষদের মাতৃভাষা কী ?,"{'text': [], 'answer_start': []}"
4,bengali--3722386795158401774-6-3048,গ্ত্সাং-পা রাজবংশ,"K.-H. Everding (2000), Das Königreich Mangyul Gungthang, Vol. I-II. Bonn.\nH. Hoffmann (1986), Tibet. A Handbook, Bloomington.\nDung-dkar blo-

In [15]:
model_checkpoint = "csebuetnlp/banglabert"
batch_size =16

In [16]:
!pip install sentencepiece -q

     |████████████████████████████████| 1.3 MB 31.8 MB/s 


In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [18]:
# check if the tokenizer is fast tokenizer

import transformers

assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [19]:
tokenizer("আপনার নাম কি?", "আমার নাম ফয়সাল")

{'input_ids': [2, 1338, 993, 817, 35, 3, 878, 993, 16310, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [20]:
max_length = 512  # The maximum length of a feature (question and context)
doc_stride = 128  # The allowed overlap between two part of the context 

In [21]:
pad_on_right = tokenizer.padding_side == "right"

In [22]:
def prepare_train_features(examples):
    
    #removing left whitespaces
    examples["question"] = [q.lstrip() for q in examples["question"]]


    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

   
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        
        sequence_ids = tokenized_examples.sequence_ids(i)

        
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

           
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [23]:
tokenized_datasets = raw_datasets.map(
    prepare_train_features, batched=True, remove_columns=raw_datasets["train"].column_names
)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [24]:
tokenized_datasets["train"].column_names

['input_ids',
 'token_type_ids',
 'attention_mask',
 'start_positions',
 'end_positions']

In [25]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at csebuetnlp/banglabert were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at csebuetnlp/banglabert and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probab

In [26]:
model.config

ElectraConfig {
  "_name_or_path": "csebuetnlp/banglabert",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.21.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

In [27]:
!pip install wandb -q

     |████████████████████████████████| 1.8 MB 36.1 MB/s 
     |████████████████████████████████| 181 kB 72.9 MB/s 
     |████████████████████████████████| 157 kB 75.1 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 157 kB 73.8 MB/s 
     |████████████████████████████████| 157 kB 75.0 MB/s 
     |████████████████████████████████| 157 kB 80.5 MB/s 
     |████████████████████████████████| 157 kB 71.7 MB/s 
     |████████████████████████████████| 156 kB 78.5 MB/s 


In [28]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [29]:
import wandb 
from google.colab import drive
import os

model_name = model_checkpoint.split("/")[-1]


drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [30]:
model_name

'banglabert'

In [31]:
!ls

gdrive	     test.json	 train.json   validaiton.jsonl
sample_data  test.jsonl  train.jsonl  validation.json


In [32]:
wandb_dir = f"/content/gdrive/MyDrive/DL_models/language_model_bn/wandb_files/"

wandb.init(dir=os.getenv("WANDB_DIR",wandb_dir),project = "language-model-bn",id ="bangla_bert_tes-1",resume="allow")

wandb: Currently logged in as: afschowdhury. Use `wandb login --relogin` to force relogin


In [46]:
# Training Arguments
args = TrainingArguments(
    seed = 42,
    output_dir = f"/content/gdrive/MyDrive/DL_models/language_model_bn/{model_name}-test-1",
    evaluation_strategy = "epoch", #default step 500
    learning_rate=2e-5,
    lr_scheduler_type = "linear",
    warmup_ratio = 0.1,
    # gradient_accumulation_steps =2,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_strategy = "epoch",
    # save_steps = 1000,
    save_total_limit=2,
    num_train_epochs=10,
    weight_decay=0.1,
    fp16=True,
    report_to = "wandb",
    run_name ="language-model-bn(qas)_bnbert"
    
)

PyTorch: setting up devices


In [34]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 2556
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 2556
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 2556
    })
})

In [35]:
from transformers import default_data_collator

data_collator = default_data_collator

In [48]:
trainer = Trainer(
    model=model,
    args = args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Using cuda_amp half precision backend


In [37]:
import torch
torch.cuda.empty_cache()

In [38]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |  430574 KB |  430574 KB |  430574 KB |       0 B  |\n|       from large pool |  430080 KB |  430080 KB |  430080 KB |       0 B  |\n|       from small pool |     494 KB |     494 KB |     494 KB |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |  430574 KB |  430574 KB |  430574 KB |       0 B  |\n|       from large pool |  430080 KB |  430080 KB |

In [49]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2556
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1600
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss
1,No log,1.306578
2,No log,0.871719
3,No log,0.796264
4,1.190200,0.687296
5,1.190200,0.618362
6,1.190200,0.604796
7,0.741300,0.599866
8,0.741300,0.597402
9,0.741300,0.066814
10,0.525500,0.029956


***** Running Evaluation *****
  Num examples = 2556
  Batch size = 16
Saving model checkpoint to /content/gdrive/MyDrive/DL_models/language_model_bn/banglabert-test-1/checkpoint-160
Configuration saved in /content/gdrive/MyDrive/DL_models/language_model_bn/banglabert-test-1/checkpoint-160/config.json
Model weights saved in /content/gdrive/MyDrive/DL_models/language_model_bn/banglabert-test-1/checkpoint-160/pytorch_model.bin
tokenizer config file saved in /content/gdrive/MyDrive/DL_models/language_model_bn/banglabert-test-1/checkpoint-160/tokenizer_config.json
Special tokens file saved in /content/gdrive/MyDrive/DL_models/language_model_bn/banglabert-test-1/checkpoint-160/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2556
  Batch size = 16
Saving model checkpoint to /content/gdrive/MyDrive/DL_models/language_model_bn/banglabert-test-1/checkpoint-320
Configuration saved in /content/gdrive/MyDrive/DL_models/language_model_bn/banglabert-test-1/checkpoint-320/conf

TrainOutput(global_step=1600, training_loss=0.7728814423084259, metrics={'train_runtime': 1347.8799, 'train_samples_per_second': 18.963, 'train_steps_per_second': 1.187, 'total_flos': 6678745102172160.0, 'train_loss': 0.7728814423084259, 'epoch': 10.0})

In [40]:
trainer.save_model(f"/content/gdrive/MyDrive/DL_models/language_model_bn/{model_name}-test-1/bnbert")

Saving model checkpoint to /content/gdrive/MyDrive/DL_models/language_model_bn/banglabert-test-1/afs_xlm_roberta
Configuration saved in /content/gdrive/MyDrive/DL_models/language_model_bn/banglabert-test-1/afs_xlm_roberta/config.json
Model weights saved in /content/gdrive/MyDrive/DL_models/language_model_bn/banglabert-test-1/afs_xlm_roberta/pytorch_model.bin
tokenizer config file saved in /content/gdrive/MyDrive/DL_models/language_model_bn/banglabert-test-1/afs_xlm_roberta/tokenizer_config.json
Special tokens file saved in /content/gdrive/MyDrive/DL_models/language_model_bn/banglabert-test-1/afs_xlm_roberta/special_tokens_map.json


### Inferencing

In [41]:
model_location = f"/content/gdrive/MyDrive/DL_models/language_model_bn/{model_name}-test-1/bnbert"

In [42]:
model_location

'/content/gdrive/MyDrive/DL_models/language_model_bn/banglabert-test-1/afs_xlm_roberta'

In [43]:
from transformers import  pipeline




nlp = pipeline('question-answering', model=model_location, tokenizer=model_location)
QA_input = {
    'question': 'আপনার নাম কি?',
    'context': 'আমার নাম দীপ সাহা। আমি ইসলামী বিশ্ববিদ্যালয়ে পড়ি'
}
res = nlp(QA_input)
res

loading configuration file /content/gdrive/MyDrive/DL_models/language_model_bn/banglabert-test-1/afs_xlm_roberta/config.json
Model config ElectraConfig {
  "_name_or_path": "/content/gdrive/MyDrive/DL_models/language_model_bn/banglabert-test-1/afs_xlm_roberta",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.21.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 3

{'score': 0.39333489537239075, 'start': 9, 'end': 17, 'answer': 'দীপ সাহা'}

In [44]:

QA_input = {
    'question': 'মেক্সিকো সিটি প্রথম কোন বছর প্যান আমেরিকা গেমস আয়োজন করেছিল?',
    'context': 'মেক্সিকো সিটি একমাত্র লাতিন আমেরিকান শহর হিসেবে অলিম্পিক গেমস আয়োজন করেছে। ১৯৬৮ সালে গ্রীষ্মকালীন অলিম্পিক আয়োজন করে তারা। (এটাও পরিবর্তন হবে রিওকে ধন্যবাদ, ২০১৬ গ্রীষ্মকালীন গেমসের আয়োজক)। শহরটি ১৯৫৫ এবং ১৯৭৫ প্যান আমেরিকান গেমস আয়োজন করে, সান্তিয়াগো এবং সাও পাওলো সরে যাওয়ার পর সর্বশেষ। আইসিএফ ফ্লাটওয়াটার রেসিং বিশ্ব চ্যাম্পিয়নশিপ এখানে ১৯৭৪ এবং ১৯৯৪ সালে অনুষ্ঠিত হয়েছিল। লুচা লিব্রে হচ্ছে মেক্সিকোর কুস্তির এক ধরন, এবং এটি সারা দেশে জনপ্রিয় খেলাগুলোর মধ্যে অন্যতম। শহরের প্রধান মাঠগুলি হল এরিনা মেক্সিকো এবং এরিনা কোলিসেও।'
    }
res = nlp(QA_input)
res

{'score': 0.12903007864952087,
 'start': 199,
 'end': 212,
 'answer': '১৯৫৫ এবং ১৯৭৫'}